In [55]:
import pandas as pd
import numpy as np
import os
import re
import nltk
#from sentence_transformers import SentenceTransformer, util
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 500)

## Заказчик<br>
ООО «ПРОСЕПТ» — российская производственная компания, специализирующаяся на выпуске профессиональной химии. В своей работе используют опыт ведущих мировых производителей и сырье крупнейших химических концернов. Производство и логистический центр расположены в непосредственной близости от Санкт-Петербурга, откуда продукция компании поставляется во все регионы России.

Сайт: https://prosept.ru/

## Описание проекта
Заказчик производит несколько сотен различных товаров бытовой и промышленной химии, а затем продаёт эти товары через дилеров. Дилеры, в свою очередь, занимаются розничной продажей товаров в крупных сетях магазинов и на онлайн площадках.
Для оценки ситуации,  управления ценами и  бизнесом в целом, заказчик периодически собирает информацию о том, как дилеры продают их товар. Для этого они парсят сайты дилеров, а затем сопоставляют товары и цены.
Зачастую описание товаров на сайтах дилеров отличаются от того описания, что даёт заказчик. Например, могут добавляться новый слова (“универсальный”, “эффективный”), объём (0.6 л -> 600 мл). Поэтому сопоставление товаров дилеров с товарами производителя делается вручную.

**Цель этого проекта**- разработка решения, которое отчасти автоматизирует процесс сопоставления товаров.<br> **Основная идея** - предлагать несколько товаров заказчика, которые с наибольшей вероятностью соответствуют размечаемому товару дилера. 
Предлагается реализовать это решение, как онлайн сервис, открываемый в веб-браузере. Выбор наиболее вероятных подсказок делается методами машинного обучения.


# Задача ML:
    -Разработка рекомендательной модели на основе данных о товарах.
    -Интеграция модели с бэкендом для предоставления вариантов соответствия.
    -Тестирование и оптимизация модели для достижения высокой точности.


## Описание данных
Заказчик предоставил несколько таблиц (дамп БД), содержащих необходимые данные:

1.	**marketing_dealer - список дилеров**
2.	**marketing_dealerprice - результат работы парсера площадок дилеров**:
    -	product_key - уникальный номер позиции;
    -	price - цена;
    -	product_url - адрес страницы, откуда собраны данные;
    -	product_name - заголовок продаваемого товара;
    -	date - дата получения информации;
    -	dealer_id - идентификатор дилера (внешний ключ к marketing_dealer)
3.	**marketing_product - список товаров, которые производит и распространяет заказчик**
    -	article - артикул товара;
    -	ean_13 - код товара (см. EAN 13)
    -	name - название товара;
    -	cost - стоимость;
    -	min_recommended_price - рекомендованная минимальная цена;
    -	recommended_price - рекомендованная цена;
    -	category_id - категория товара;
    -	ozon_name - названиет товара на Озоне;
    -	name_1c - название товара в 1C;
    -	wb_name - название товара на Wildberries;
    -	ozon_article - описание для Озон;
    -	wb_article - артикул для Wildberries;
    -	ym_article - артикул для Яндекс.Маркета;

4.	**marketing_productdealerkey - таблица матчинга товаров заказчика и товаров дилеров**
    -	key - внешний ключ к marketing_dealerprice
    -	product_id - внешний ключ к marketing_product
    -	dealer_id - внешний ключ к marketing_dealer


# Чтение данных:

In [56]:
path = '../Хакатон/данные/'

In [57]:
os.listdir('../Хакатон/данные')

['marketing_productdealerkey.csv',
 'marketing_dealer.csv',
 'marketing_dealerprice.csv',
 'marketing_product.csv',
 'ТЗ Просепт.docx']

### marketing_dealer - список дилеров

In [58]:
marketing_dealer = pd.read_csv(path + 'marketing_dealer.csv', engine='python', sep = ';')

In [59]:
marketing_dealer.sort_values(by='id')

,id,name
0,1,Moi_vibor_WB
1,2,Akson
2,3,Bafus
14,4,Baucenter
3,5,Castorama
4,6,Cubatora
5,7,Komus
15,8,Leroy_Merlin
6,9,Megastroy
7,10,OnlineTrade


Имеем 18 айдишников от 1 до 18

**marketing_dealerprice - результат работы парсера площадок дилеров**:
   -	product_key - уникальный номер позиции;
   -	price - цена;
   -	product_url - адрес страницы, откуда собраны данные;
   -	product_name - заголовок продаваемого товара;
   -	date - дата получения информации;
   -	dealer_id - идентификатор дилера (внешний ключ к marketing_dealer)

In [60]:
marketing_dealerprice = pd.read_csv(path + 'marketing_dealerprice.csv', engine='python', sep = ';')

In [61]:
marketing_dealerprice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20416 entries, 0 to 20415
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            20416 non-null  int64  
 1   product_key   20416 non-null  object 
 2   price         20416 non-null  float64
 3   product_url   20182 non-null  object 
 4   product_name  20416 non-null  object 
 5   date          20416 non-null  object 
 6   dealer_id     20416 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 1.1+ MB


In [62]:
marketing_dealerprice.head()

,id,product_key,price,product_url,product_name,date,dealer_id
0,2,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_prosept_universal_spray_500ml/,"Средство универсальное Prosept Universal Spray, 500мл",2023-07-11,2
1,3,546408,175.0,https://akson.ru//p/kontsentrat_prosept_multipower_dlya_mytya_polov_tsitrus_1l/,"Концентрат Prosept Multipower для мытья полов, цитрус 1л",2023-07-11,2
2,4,546234,285.0,https://akson.ru//p/sredstvo_dlya_chistki_lyustr_prosept_universal_anti_dust_500ml/,"Средство для чистки люстр Prosept Universal Anti-dust, 500мл",2023-07-11,2
3,5,651258,362.0,https://akson.ru//p/udalitel_rzhavchiny_prosept_rust_remover_0_5l_023_05/,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 023-05",2023-07-11,2
4,6,546355,205.0,https://akson.ru//p/sredstvo_moyushchee_dlya_bani_i_sauny_prosept_multipower_wood_1l/,Средство моющее для бани и сауны Prosept Multipower Wood 1л,2023-07-11,2


In [63]:
marketing_dealerprice[marketing_dealerprice['product_key'] == '546227']

,id,product_key,price,product_url,product_name,date,dealer_id
0,2,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_prosept_universal_spray_500ml/,"Средство универсальное Prosept Universal Spray, 500мл",2023-07-11,2
594,592,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_prosept_universal_spray_500ml/,"Средство универсальное Prosept Universal Spray, 500мл",2023-07-11,2
1992,1992,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_prosept_universal_spray_500ml/,"Средство универсальное Prosept Universal Spray, 500мл",2023-07-12,2
3608,3730,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_prosept_universal_spray_500ml/,"Средство универсальное Prosept Universal Spray, 500мл",2023-07-13,2
5136,5277,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_prosept_universal_spray_500ml/,"Средство универсальное Prosept Universal Spray, 500мл",2023-07-14,2
5203,5344,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_prosept_universal_spray_500ml/,"Средство универсальное Prosept Universal Spray, 500мл",2023-07-14,2


**marketing_product - список товаров, которые производит и распространяет заказчик**:
- article - артикул товара;
- ean_13 - код товара (см. EAN 13)
- name - название товара;
- cost - стоимость;
- min_recommended_price - рекомендованная минимальная цена;
- recommended_price - рекомендованная цена;
- category_id - категория товара;
- ozon_name - названиет товара на Озоне;
- name_1c - название товара в 1C;
- wb_name - название товара на Wildberries;
- ozon_article - описание для Озон;
- wb_article - артикул для Wildberries;
- ym_article - артикул для Яндекс.Маркета;

In [64]:
marketing_product= pd.read_csv(path + 'marketing_product.csv', engine='python', sep = ';', index_col= 0)

In [65]:
marketing_product.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496 entries, 0 to 495
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 496 non-null    int64  
 1   article            496 non-null    object 
 2   ean_13             464 non-null    float64
 3   name               494 non-null    object 
 4   cost               491 non-null    float64
 5   recommended_price  491 non-null    float64
 6   category_id        447 non-null    float64
 7   ozon_name          458 non-null    object 
 8   name_1c            485 non-null    object 
 9   wb_name            455 non-null    object 
 10  ozon_article       365 non-null    float64
 11  wb_article         340 non-null    float64
 12  ym_article         337 non-null    object 
 13  wb_article_td      32 non-null     object 
dtypes: float64(6), int64(1), object(7)
memory usage: 58.1+ KB


In [66]:
marketing_product.head()

,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
0,245,008-1,4.680008e+12,Антисептик невымываемыйPROSEPT ULTRAконцентрат 1:10 / 1 л,360.00,858.0,20.0,"Антисептик невымываемый для ответственных конструкций PROSEPT ULTRA, концентрат, 1 л.","Антисептик невымываемый для ответственных конструкций PROSEPT ULTRA, концентрат, 1 л.","Антисептик невымываемый для ответственных конструкций PROSEPT ULTRA, концентрат, 1 л.",189522705.0,150033482.0,008-1,NaN
1,3,242-12,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,460.16,1075.0,NaN,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,NaN,NaN,NaN,NaN,NaN
2,443,0024-06 с,4.680008e+12,"Герметик акриловый цвет сосна, ф/п 600мл",307.00,644.0,25.0,"Герметик акриловый для швов для деревянных домов, конструкций, изделий PROSEPT цвет сосна, ф/п 600мл","Герметик акриловый цвет сосна, ф/п 600мл","Герметик акриловый для швов для деревянных домов, конструкций, изделий PROSEPT цвет сосна, ф/п 600мл",189522735.0,150126217.0,0024-06-с,NaN
3,147,305-2,4.610093e+12,Кондиционер для белья с ароматом королевского ИрисаCrystal Rinserконцентрат / 2 л,157.73,342.0,29.0,"Кондиционер для белья ""Королевский Ирис"" Prosept Crystal Rinser, 2 л.","Кондиционер для белья ""Королевский Ирис"" Prosept Crystal Rinser, 2 л.","Кондиционер для белья ""Королевский Ирис"" Prosept Crystal Rinser, 2 л.",339377922.0,150032962.0,305-2,NaN
4,502,0024-7 б,NaN,"Герметик акриловой цвет Белый, 7 кг",NaN,NaN,NaN,NaN,NaN,NaN,189522867.0,150126216.0,0024-7-б,NaN


In [67]:
i = 15
print('Название продукта у производителя')
print(f'{marketing_product["name"][i]}')
print('------------------------------------')
print('Название продукта на озоне')
print(f'{marketing_product["ozon_name"][i]}')
print('------------------------------------')
print('Название продукта на вайлдбериз')
print(f'{marketing_product["wb_name"][i]}')
print('------------------------------------')
print('Название продукта в 1С')
print(f'{marketing_product["name_1c"][i]}')

Название продукта у производителя
Средство для мытья полов и стенMultipower "Полевые цветы"концентрат  1:10 – 1:120 / 0,8 л
------------------------------------
Название продукта на озоне
nan
------------------------------------
Название продукта на вайлдбериз
nan
------------------------------------
Название продукта в 1С
Средство для мытья полов и стенMultipower "Полевые цветы"концентрат  1:10 – 1:120 / 0,8 л


In [68]:
marketing_product.isnull().sum()

id                     0
article                0
ean_13                32
name                   2
cost                   5
recommended_price      5
category_id           49
ozon_name             38
name_1c               11
wb_name               41
ozon_article         131
wb_article           156
ym_article           159
wb_article_td        464
dtype: int64

In [69]:
marketing_product = marketing_product.dropna(subset='name')

In [70]:
marketing_product.fillna({'ozon_name' : 'Товар не представлен',  'wb_name': 'Товар не представлен'}, inplace=True)

In [71]:
# заполним пропуски в marketing_name данными из столбца name
marketing_product['name_1c'].fillna(marketing_product['name'], inplace=True)

## Добавим новый признак- объем вещества, выделив его из описания.

In [72]:
# создадим столбец с единицами измерения, указанные в названии товара, и уберём их из самих названий
def mesuares(text):    
    pattern = r'(\d+\s?[лмкг]+)'
    match  = re.search(pattern, text)
    if match:
        text = match.group(1)
        text = text.replace(pattern, '')
    return text

In [73]:
#marketing_product['mesuares'] = marketing_product['name'].apply(mesuares)
#marketing_dealerprice['mesuares'] = marketing_dealerprice['product_name'].apply(mesuares)

In [74]:
marketing_product.loc[marketing_product['id'] ==245, 'name']

0    Антисептик невымываемыйPROSEPT ULTRAконцентрат 1:10  / 1 л
Name: name, dtype: object

In [75]:
display(marketing_product.head(2))
display(marketing_dealerprice.head(2))

,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
0,245,008-1,4.680008e+12,Антисептик невымываемыйPROSEPT ULTRAконцентрат 1:10 / 1 л,360.00,858.0,20.0,"Антисептик невымываемый для ответственных конструкций PROSEPT ULTRA, концентрат, 1 л.","Антисептик невымываемый для ответственных конструкций PROSEPT ULTRA, концентрат, 1 л.","Антисептик невымываемый для ответственных конструкций PROSEPT ULTRA, концентрат, 1 л.",189522705.0,150033482.0,008-1,NaN
1,3,242-12,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,460.16,1075.0,NaN,Товар не представлен,Антигололед - 32 PROSEPTготовый состав / 12 кг,Товар не представлен,NaN,NaN,NaN,NaN


,id,product_key,price,product_url,product_name,date,dealer_id
0,2,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_prosept_universal_spray_500ml/,"Средство универсальное Prosept Universal Spray, 500мл",2023-07-11,2
1,3,546408,175.0,https://akson.ru//p/kontsentrat_prosept_multipower_dlya_mytya_polov_tsitrus_1l/,"Концентрат Prosept Multipower для мытья полов, цитрус 1л",2023-07-11,2


## Обработка текста.

In [76]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    # отделение английских слов
    pattern = re.compile(r'(?<=[а-яА-Я])(?=[A-Z])|(?<=[a-zA-Z])(?=[а-яА-Я])')
    text = re.sub(pattern, ' ', text)
    # приведение к нижнему регистру 
    text = text.lower()
    # удаление символов
    #text = re.sub(r'\W', ' ', str(text))
    # удаление одиноко стоящих слов
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    # соотношения объемов 
    pattern2 = re.compile(r'\b\d+:\d+\s*-\s*\d+:\d+\b|\s*\d+:\d+\s*')
    text = re.sub(pattern2, ' ', text)
    return "".join(lemmatizer.lemmatize(text)) 

In [77]:
marketing_dealerprice['product_name_lem'] = marketing_dealerprice['product_name'].apply(lemmatize_text)
marketing_product['name_lem'] = marketing_product['name'].apply(lemmatize_text)

In [78]:
marketing_dealerprice['product_name_lem'][0]

'средство универсальное prosept universal spray, 500мл'

In [91]:
df_1 = marketing_dealerprice[['product_name_lem']]
df_1 = df_1.rename(columns={'product_name_lem': 'name'})
df_2 = marketing_product[['name_lem']]
df_2 = df_2.rename(columns={'name_lem': 'name'})
print(df_1.head())
print('-'*100)
print(df_2.head())
print('-'*100)
print(df_1.shape)
print(df_2.shape)
df = pd.concat([df_1, df_2])

                                                           name
0         средство универсальное prosept universal spray, 500мл
1      концентрат prosept multipower для мытья полов, цитрус 1л
2  средство для чистки люстр prosept universal anti-dust, 500мл
3           удалитель ржавчины prosept rust remover 0,5л 023-05
4   средство моющее для бани и сауны prosept multipower wood 1л
----------------------------------------------------------------------------------------------------
                                                                                    name
0                                 антисептик невымываемый prosept ultra концентрат / 1 л
1                                        антигололед - 32 prosept готовый состав / 12 кг
2                                               герметик акриловый цвет сосна, ф/п 600мл
3  кондиционер для белья с ароматом  королевского ириса crystal rinser концентрат / 2 л 
4                                                   герметик акриловой

In [92]:
df.head()

,name
0,"средство универсальное prosept universal spray, 500мл"
1,"концентрат prosept multipower для мытья полов, цитрус 1л"
2,"средство для чистки люстр prosept universal anti-dust, 500мл"
3,"удалитель ржавчины prosept rust remover 0,5л 023-05"
4,средство моющее для бани и сауны prosept multipower wood 1л


In [93]:
df.shape

(20910, 1)

**marketing_productdealerkey - таблица матчинга товаров заказчика и товаров дилеров**
   - key - внешний ключ к marketing_dealerprice
   - product_id - внешний ключ к marketing_product
   - dealer_id - внешний ключ к marketing_dealer

In [82]:
marketing_productdealerkey = pd.read_csv(path + 'marketing_productdealerkey.csv', 
                                         engine='python', sep = ';')

In [83]:
marketing_productdealerkey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1700 non-null   int64 
 1   key         1700 non-null   object
 2   dealer_id   1700 non-null   int64 
 3   product_id  1700 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 53.2+ KB


In [84]:
marketing_productdealerkey.head()

,id,key,dealer_id,product_id
0,1,546227,2,12
1,2,651265,2,106
2,3,546257,2,200
3,4,546408,2,38
4,5,651258,2,403


In [85]:
marketing_productdealerkey['key'].unique()

array(['546227', '651265', '546257', ..., '534659036', '898350801',
       '534666258'], dtype=object)

## Векторизация

In [90]:
count_tf_idf = TfidfVectorizer()
df = count_tf_idf.fit_transform(df['name'])
df_1 = count_tf_idf.transform(df_1['name'])
df_2 = count_tf_idf.transform(df_2['name'])

In [ ]:
print(df_1.shape)
print(df_2.shape)

(20416, 1115)
(494, 1115)


In [ ]:
df_1 = df_1.toarray()
df_2 = df_2.toarray()

In [ ]:
print(type(df_1))
print(type(df_2))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
print(df_1.shape)
print(df_2.shape)

(20416, 1115)
(494, 1115)


In [ ]:
pairwise_distances(df_1, df_2, 'jaccard').shape

(20416, 494)

In [ ]:
df = pd.DataFrame(index = marketing_product['id'], 
                    columns = marketing_dealerprice['product_key']+ '_' + pd.Series(range(marketing_dealerprice.shape[0])).astype(str), 
                    data = pairwise_distances(df_2,df_1 , metric = 'cosine'))

In [ ]:
df.head()

,546227_0,546408_1,546234_2,651258_3,546355_4,831859_5,546406_6,831858_7,857015_8,651265_9,...,590939046_20406,534463306_20407,528623838_20408,534580937_20409,685030286_20410,534517451_20411,530308963_20412,531730388_20413,1090913025_20414,1077090171_20415
id,,,,,,,,,,,,,,,,,,,,,
245,0.965515,0.874883,0.974378,0.976616,0.973034,0.878684,0.975320,0.876570,0.979191,0.980206,...,0.976516,0.976877,0.799277,0.978000,0.980436,0.971010,0.908137,0.966876,1.000000,0.983379
3,0.983547,0.984236,0.987776,0.988843,0.987135,0.984715,0.988225,0.984449,0.944947,0.990556,...,0.819218,0.859148,0.942240,0.865990,0.990666,0.823412,1.000000,0.984197,0.864155,0.956027
443,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.743234,1.000000
147,1.000000,0.946363,0.989531,1.000000,0.988982,0.947992,0.989916,0.947086,1.000000,0.991913,...,1.000000,0.990552,0.969974,0.991011,1.000000,0.988155,1.000000,0.986466,0.992868,0.993209
502,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.950412,1.000000,...,0.944037,1.000000,0.955923,1.000000,1.000000,1.000000,1.000000,1.000000,0.687739,0.960392


### Ввод названия товара дилера с возвратом соответствующих названий у заказчика.

In [ ]:
marketing_dealerprice.loc[marketing_dealerprice['product_key'] == '530308963', 'product_name']

16924    антисептик многофункциональный фбс, гост 5 л
18544    антисептик многофункциональный фбс, гост 5 л
20412    антисептик многофункциональный фбс, гост 5 л
Name: product_name, dtype: object

In [ ]:
name = 'Антисептик многофункциональный ФБС, ГОСТ 5 л'

In [ ]:
marketing_dealerprice.loc[marketing_dealerprice['product_name'] == name, 'product_key']

16924    530308963
18544    530308963
20412    530308963
Name: product_key, dtype: object

In [ ]:
product_key = marketing_dealerprice.loc[marketing_dealerprice['product_name'] == name, 'product_key'].to_list()[0]
#print(df.iloc[[product_key]].sort_values()[:5])

In [ ]:
product_key

'530308963'

In [ ]:
top_k = 5

Тут косяк.
С индексом вывод названия, с полученным ключем ошибка

In [ ]:
z = df.iloc[:, 1].sort_values()[:top_k].index.to_list()

In [ ]:
marketing_product.loc[marketing_product['id'].isin(z) , 'name']

65                          концентрат для мытья полов и стен multipower "цитрус" 0,8 л
91                     концентрат для мытья полов и стен multipower "цитрус" 0,8 л 2 шт
176            концентрат для мытья полов и стен multipower "цитрус" концентрат / 0,8 л
269    концентрат эконом-класса для мытья полов multipower (цитрус)  концентрат - / 5 л
270    концентрат эконом-класса для мытья полов multipower (цитрус)  концентрат - / 1 л
Name: name, dtype: object

id
245    0.874883
3      0.984236
443    1.000000
147    0.946363
502    1.000000
         ...   
127    0.946995
160    0.948438
74     0.986203
34     0.664414
52     0.945383
Name: 546408_1, Length: 494, dtype: float64

### Подсчет метрики accuracy

In [ ]:
match_df = df[list(set(df.columns))]

df.columns = df.columns.astype(str)

matches = []

for col in set(df.columns):
        
    top_cands = df.loc[:, col].sort_values(ascending=True)[:top_k].index.tolist()
    product_key = ''.join(col.split('_')[:-1])
    
    # print(product_key)
    
    if  marketing_productdealerkey.loc[marketing_productdealerkey['key'] == product_key].shape[0] == 0:
        matches.append(0)
        continue
        
    match_id = marketing_productdealerkey.loc[marketing_productdealerkey['key'] == product_key, 'product_id'].values[0]
    if match_id in top_cands:
        matches.append(1)
    else:
        matches.append(0)
        
np.mean(matches)

0.7446120689655172